# Load Data

> Functions that load the data for the map.   

In [ ]:
#| default_exp load_data

In [ ]:
#| export
from nbdev.showdoc import *
import geopandas, pandas
from git import Repo

In [ ]:
#| hide
repo = Repo('.', search_parent_directories=True)
fp = str(repo.working_tree_dir) + "/testData/"

# Check that the files exist using fastcore

In [ ]:
#| export
class SolomonGeo:
    '''
    Load the solomon islands geographies
    '''
    def __init__(self):
        repo = Repo('.', search_parent_directories=True)
        fp = str(repo.working_tree_dir) + "/testData/"
        adm3 = geopandas.read_file('zip://' +fp + 'slb_admbnda_adm3.zip')
        adm3.to_file('myJson.geojson', driver='GeoJSON')

        # 
        self.adm3 = adm3
    # TODO create functions to return 

In [ ]:
show_doc(SolomonGeo)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/load_data.py#L12){target="_blank" style="float:right; font-size:smaller"}

### SolomonGeo

>      SolomonGeo ()

Load the solomon islands geographies

## Testing

In [ ]:
sol_geo = SolomonGeo()
sol_geo.adm3.head()

ERROR 1: PROJ: proj_create_from_database: Open of /home/tom/mambaforge/share/proj failed


,OBJECTID,SINSO_WID,ADM3_PCODE,ADM3_NAME,ADM2_PCODE,ADM2_NAME,ADM1_PCODE,ADM1_NAME,ADM0_PCODE,COUNTRY,SINSO_CID,SINSO_PID,geometry
0,1,1001,SB1010431001,Nggosi,SB101043,West Honiara,SB10,Honiara,SB,Solomon Islands,1043,10,"POLYGON Z ((1104818.130 -1046772.426 0.000, 11..."
1,2,1002,SB1010431002,Mbumburu,SB101043,West Honiara,SB10,Honiara,SB,Solomon Islands,1043,10,"POLYGON Z ((1107079.803 -1048040.379 0.000, 11..."
2,3,1003,SB1010431003,Rove - Lengakiki,SB101043,West Honiara,SB10,Honiara,SB,Solomon Islands,1043,10,"POLYGON Z ((1106918.942 -1047270.637 0.000, 11..."
3,4,1004,SB1010421004,Cruz,SB101042,Central Honiara,SB10,Honiara,SB,Solomon Islands,1042,10,"POLYGON Z ((1107949.631 -1047995.867 0.000, 11..."
4,5,1005,SB1010421005,Vavaea,SB101042,Central Honiara,SB10,Honiara,SB,Solomon Islands,1042,10,"POLYGON Z ((1108708.612 -1047377.358 0.000, 11..."


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()